<a href="https://colab.research.google.com/github/vivekgautamgv/Python-For-Finance/blob/main/crypto_return_168.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

# Define parameters
symbol = 'BTC-USD'  # Crypto symbol
start_date = '2020-01-01'
end_date = '2024-01-01'
short_window = 50
long_window = 200

# Download historical data
data = yf.download(symbol, start=start_date, end=end_date)
data['Short_MA'] = data['Close'].rolling(window=short_window, min_periods=1).mean()
data['Long_MA'] = data['Close'].rolling(window=long_window, min_periods=1).mean()

# Generate trading signals
data['Signal'] = 0
data['Signal'][short_window:] = np.where(data['Short_MA'][short_window:] > data['Long_MA'][short_window:], 1, 0)
data['Position'] = data['Signal'].diff()

# Backtesting the strategy
initial_capital = 10000
shares = 0
capital = initial_capital
position = 0
equity = []
trade_count = 0  # To count the number of trades

for i in range(len(data)):
    if data['Position'][i] == 1:
        # Buy
        shares = capital / data['Close'][i]
        position = shares * data['Close'][i]
        capital = 0
        trade_count += 1  # Increment trade count on buy
    elif data['Position'][i] == -1:
        # Sell
        capital = shares * data['Close'][i]
        shares = 0
        position = 0
        trade_count += 1  # Increment trade count on sell
    equity.append(capital + position)

data['Equity'] = equity

# Calculate returns and drawdown
data['Returns'] = data['Equity'].pct_change()
data['Cumulative_Returns'] = (1 + data['Returns']).cumprod() - 1
data['Drawdown'] = data['Equity'] / data['Equity'].cummax() - 1

# Plot results
plt.figure(figsize=(14, 7))
plt.subplot(2, 1, 1)
plt.plot(data.index, data['Close'], label='Close Price')
plt.plot(data.index, data['Short_MA'], label='50-Day MA')
plt.plot(data.index, data['Long_MA'], label='200-Day MA')
plt.title('Crypto Price and Moving Averages')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(data.index, data['Cumulative_Returns'], label='Cumulative Returns')
plt.plot(data.index, data['Drawdown'], label='Drawdown')
plt.title('Strategy Performance')
plt.legend()

plt.tight_layout()
plt.show()

# Print final results
final_return_percentage = data['Cumulative_Returns'].iloc[-1] * 100
final_equity = data['Equity'].iloc[-1]
max_drawdown = data['Drawdown'].min()

print("Total Invested amount : ",initial_capital)
print(f"Final Return Percentage: {final_return_percentage:.2f}%")
print(f"Final Equity: ${final_equity:.2f}")
print(f"Total Trades: {trade_count}")
print(f"Max Drawdown: {-max_drawdown * 100:.2f}%")
